In [2]:

import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import sys

input_directory = '../data/raw/img_align_celeba/'

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

from src.utils.utils import load_and_preprocess_image
load_and_preprocess_image(input_directory, scale_factor=4)

In [3]:
from src.models.model import build_sr_model
# Build the model
model = build_sr_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 64)    1792      
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 64)    36928     
                                                                 
 up_sampling2d (UpSampling2  (None, None, None, 64)    0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, None, None, 64)    36928     
                                                                 
 conv2d_3 (Conv2D)           (None, None, None, 3)     1731      
                                                             

In [4]:
model.compile(optimizer='adam', loss='mean_squared_error')

: 

In [ ]:
# Example usage in training
image_directory = 'data/raw/img_align_celeba/'
batch_size = 16
scale_factor = 4

# Use the generator function in your model's fit method
model.fit(
    generate_batches(image_directory, batch_size, scale_factor),
    steps_per_epoch=len(os.listdir(image_directory)) // batch_size,
    epochs=10
)

In [5]:
from src.utils.utils import load_image_data

lr_dir = '../data/processed/LR/'
hr_dir = '../data/processed/HR/'

lr_images, hr_images = load_image_data(lr_dir, hr_dir)